In [1]:
# Parameters 
import yfinance as yf
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt


def download_data_1Yr(tickers, period="1y"):
    data = yf.download(tickers, period=period)
    #time.sleep(5)
    return data['Adj Close']  # Getting the adjusted close prices

def download_data_5Yr(tickers, period="5y"):
    data = yf.download(tickers, period=period)
    #time.sleep(5)
    return data['Adj Close']  # Getting the adjusted close prices

#Set which tickers we want 
tickers = ["BNO", "USO"]
dataset = download_data_1Yr(tickers) #Downloads dataset
dataset.to_csv("ticker_data.csv")


[*********************100%%**********************]  2 of 2 completed


In [6]:
# Check if the required columns exist in the dataframe
if 'BNO' in dataset.columns and 'USO' in dataset.columns:
    # Creating a new column 'Difference' which is the difference between 'BNO' and 'USO'
    dataset['Spread'] = dataset['USO'] - dataset['BNO']
else:
    print("Columns BNO and/or USO do not exist in the dataframe.")
        
# Find StdDev 
std_Spread = dataset['Spread'].std()
print(f"STD.DEV= {std_Spread}")

# Find Mean
spread_mean = dataset['Spread'].mean()
print(f"Mean= {spread_mean}")

# Checking for NaN values before calculating the correlation
if dataset['BNO'].notna().all() and dataset['USO'].notna().all():
    # Calculating the correlation between 'BNO' and 'USO' columns
    correlation_bno_uso = dataset['BNO'].corr(dataset['USO'])
    print(f"Correlation= {correlation_bno_uso}")
else:
    print("There are NaN values in the 'BNO' and/or 'USO'. Correlation calculation skipped.")

STD.DEV= 3.452317040340698
Mean= 41.522698500799756
Correlation= 0.9821866732758846


In [30]:
#Set thresholds based on statistical significance, such as a z-score of 1.5 or -1.5 as trigger points.
#Set Vars
Z_ScoreTrigger = 1.0

Z_ScoreExit = 0 #exits when asset at this Z-score

dataset['Spread_ZScore'] = (dataset['Spread'] - spread_mean) / std_Spread


# Create signals for trading (1 for long the spread, -1 for short the spread, 0 for no position)
long_entries = (dataset['Spread_ZScore'] <= -Z_ScoreTrigger).astype(int)
short_entries = (dataset['Spread_ZScore'] >= Z_ScoreTrigger).astype(int)
exits = ((dataset['Spread_ZScore'] >= -Z_ScoreExit) & (dataset['Spread_ZScore'] <= Z_ScoreExit)).astype(int)

# Combine the signals to define the positions
positions = short_entries - long_entries
positions[exits == 1] = 0
positions_df = positions.to_frame(name='positions')

# Output the positions as a trading signal
# In a live trading scenario, you'd connect this logic to your brokerage's API to execute trades.
#print(positions)

#When Positions = 1, Long the spread, Long USO, Short BNO 
if isinstance(positions, pd.Series):
    dataset['positions'] = positions  # Add positions as a column to the dataset DataFrame if it's a Series

# Now, create the 'BNO' and 'USO' columns based on the 'positions' column
# Create 'BNO' and 'USO' columns based on the conditions provided.
# Apply the logic to create 'BNO' and 'USO' columns in 'positions_df' DataFrame
positions_df['BNO.Status'] = positions_df['positions'].apply(lambda x: 'SHORT' if x == 1 else ('LONG' if x == -1 else '-'))
positions_df['USO.Status'] = positions_df['positions'].apply(lambda x: 'LONG' if x == 1 else ('SHORT' if x == -1 else '-'))
# positions_df



In [31]:
#Combine dfs, 
combined_df = pd.merge(dataset, positions_df, on='Date', how='inner')
# Drop the 'Spread' and 'Spread_ZScore' columns
combined_df = combined_df.drop(['Spread', 'Spread_ZScore','positions_x','positions_y'], axis=1)


,BNO,USO,BNO.Status,USO.Status
Date,,,,
2022-11-02,31.190001,74.080002,-,-
2022-11-03,30.860001,73.290001,-,-
2022-11-04,32.209999,76.820000,-,-
2022-11-07,32.040001,76.559998,-,-
2022-11-08,31.209999,74.470001,-,-
...,...,...,...,...
2023-10-27,31.440001,78.360001,SHORT,LONG
2023-10-30,30.680000,76.050003,SHORT,LONG
2023-10-31,30.200001,75.019997,-,-


In [32]:
combined_df

,BNO,USO,BNO.Status,USO.Status
Date,,,,
2022-11-02,31.190001,74.080002,-,-
2022-11-03,30.860001,73.290001,-,-
2022-11-04,32.209999,76.820000,-,-
2022-11-07,32.040001,76.559998,-,-
2022-11-08,31.209999,74.470001,-,-
...,...,...,...,...
2023-10-27,31.440001,78.360001,SHORT,LONG
2023-10-30,30.680000,76.050003,SHORT,LONG
2023-10-31,30.200001,75.019997,-,-


In [35]:
# Sort the dataframe by date just in case it's not sorted
combined_df.sort_values('Date', inplace=True)

# Shift the 'BNO' column up to get the next day's price in the current row
combined_df['Next_Day_BNO'] = combined_df['BNO'].shift(-1)

# Calculate the change in price
combined_df['Price_Change_BNO'] = combined_df['Next_Day_BNO'] - combined_df['BNO']

# Define a function to calculate P&L based on status
def calculate_pnl(row):
    if row['BNO.Status'] == 'LONG':
        # If 'Long', P&L is the price change
        return row['Price_Change_BNO-BNO']
    elif row['BNO.Status'] == 'SHORT':
        # If 'Short', P&L is also the price change but may be calculated differently
        return -row['Price_Change_BNO-BNO']
    else:
        # If not 'Long', no position is taken, hence P&L is 0
        return 0

# Apply the function to calculate P&L
combined_df['BNO.P&L'] = combined_df.apply(calculate_pnl, axis=1)

# Now you can drop the 'Next_Day_BNO' and 'Price_Change' columns if they are not needed
#combined_df.drop(columns=['Next_Day_BNO', 'Price_Change'], inplace=True)

combined_df

KeyError: 'Price_Change_BNO-BNO'